# Mod 03b: SparkSQL (Transformations 02)

# Lab 01: Date and TimeStamps

In [0]:
DDLSchema = "os string, event string, ts float, source string, user integer"

In [0]:
# Notice 'ts' column is in Unix Time
df = spark.read.option("header", True).schema(DDLSchema).csv("dbfs:/FileStore/tables/webevent.csv")
df.printSchema()
display(df)

In [0]:
# Cast Unix time to Python Timestamp

from pyspark.sql.functions import col
from pyspark.sql.types import *

tsDF = df.withColumn("newTS", (col("ts") / 1000000).cast("timestamp"))
display(tsDF)
tsDF.printSchema()

###'date_add' adds days to a date

In [0]:
from pyspark.sql.functions import date_add

plus_3_daysDF = tsDF.select ("newTS").withColumn("plus_3_days", date_add(col("newTS"), 3))
display(plus_3_daysDF)

### 'to_date' extracts just Year-Month-Day from timestamp

In [0]:
from pyspark.sql.functions import to_date

df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['col1'])
display(df.select(to_date(df.col1).alias('dt')))

### 'date_format' allows you to manipulate Date as desired

In [0]:
from pyspark.sql.functions import *

formatDF = (tsDF.select ("os", "event", "newTS")
                .withColumn("date_string ", date_format("newTS", "MMMM dd, yyyy"))
                .withColumn("time string", date_format("newTS", "HH:mm:ss.SSSSSS"))
           )

display(formatDF)

### Extract Year, Month, DayOfWeek, Minute, Second

In [0]:
from pyspark.sql.functions import year, month, dayofweek, minute, second

extractDF = (tsDF.withColumn("year", year(col("newTS")))
  .withColumn("month", month(col("newTS")))
  .withColumn("dayofweek", dayofweek(col("newTS")))
  .withColumn("minute", minute(col("newTS")))
  .withColumn("second", second(col("newTS")))
)
display(extractDF)

# Lab 02: Aggregations and Joins
## 'groupBy', 'agg', 'count', 'min', 'max', 'sum', 'avg'

In [0]:
deptDF = spark.read.format("parquet").load("dbfs:/FileStore/tables/dept_snappy.parquet/")
deptDF.createOrReplaceTempView("dept_tbl")
display(deptDF)

In [0]:
empDF = spark.read.format("parquet").load("dbfs:/FileStore/tables/emp_snappy.parquet/")
empDF.createOrReplaceTempView("emp_tbl")
display(empDF)

In [0]:
# 'groupBy' and 'count'
# How many rows per Dept where Dept > 400.  Sort in Descending order by Dept
(display(empDF.select("last_name", "dept", "salary")
              .filter("dept > 400")
              .groupBy("dept").count()
              .orderBy("dept", ascending=False)))

In [0]:
# DataFrame query
# For each 'Dept',  do multiple aggregate. Want Count # of row per 'Dept', Max 'salary' per 'Dept'
empDF.select("dept", "salary").groupBy("dept").agg({"*": "count", "salary": "max"}).show()

# SQL equivalent
spark.sql("select dept, count(*) as ct, max(salary) as max_sal from emp_tbl group by dept").show()


In [0]:
# Inner JOIN on empDF and deptDF
display(empDF.join(deptDF, empDF.dept == deptDF.dept, "inner"))

In [0]:
# Change Dept 100 to 999 for next Lab
# Dept 999 will now exist in empDF2, but not deptDF
from pyspark.sql.functions import when

empDF2 = empDF.withColumn("dept", when (empDF["dept"] == 100, 999).otherwise(empDF["dept"]))
empDF3 = empDF2.distinct()

In [0]:
# Left Outer Join
display(empDF3.join(deptDF, "dept", "left").orderBy(["dept"], asc = True))

In [0]:
# Join on 2 columns
display(empDF3.join(deptDF, (empDF3.dept == deptDF.dept) & (empDF3.mgr == deptDF.mgr)))

# Lab 03: Widgets: Parameterize your Queries

In [0]:
# 4 different Widget types
dbutils.widgets.text("Text", "Hello World!")
dbutils.widgets.dropdown("Dropdown", "1", [str(x) for x in range(1,10)])
dbutils.widgets.combobox("Combobox", "A", ["A", "B", "C"])
dbutils.widgets.multiselect("Multiselect", "Yes", ["Yes", "No", "Maybe"])

In [0]:
# Remove all Widgets
dbutils.widgets.removeAll()

In [0]:
display(empDF)

In [0]:
%sql
-- Notice in upper left-hand corner, parameter appears
CREATE widget TEXT last_name DEFAULT "Brown"

In [0]:
%sql
-- Run 'as is', then change Parameter in upper left-hand corner = Johnson
SELECT * FROM emp_tbl
WHERE last_name = getArgument("last_name")

In [0]:
%sql
REMOVE WIDGET last_name

# Lab 04: User Defined Functions (UDFs)
#### https://stackoverflow.com/questions/38296609/spark-functions-vs-udf-performance
#### https://towardsdatascience.com/performance-in-apache-spark-benchmark-9-different-techniques-955d3cc93266

## Comparing Performance of Normal UDF (Slow) vs. Pandas UDF (Faster)

In [0]:
# First create Schema for DataFrame
from pyspark.sql.types import LongType, TimestampType, StringType, StructType, StructField

userDefinedSchema = StructType([
  StructField("id", LongType(), True),
  StructField("location", StringType(), True),
  StructField("ts", TimestampType(), True),
  StructField("comment", StringType(), True),
  StructField("author", StringType(), True)
])     

In [0]:
# Create DataFrame
# 192 files * 28.5MB/each (24.5MB + 4MB)  Can Pack 4 files/Partition = 114MB    192/(4 files/Partition) = 48 Partitions

wikiCSVDF = (spark.read
                  .option("sep", "\t")
                  .schema(userDefinedSchema)
                  .csv("dbfs:/databricks-datasets/wiki/") 
            )
wikiCSVDF.printSchema()
wikiCSVDF.count()

In [0]:
# Row at a time using 'Normal' UDF (1 of 2)
# UDF to increment all values by 1

from pyspark.sql.functions import udf

# Use udf to define a row-at-a-time udf
@udf('long')
# Input/output are both a single double value
def plus_one(v):
      return v + 1
  
display(wikiCSVDF.withColumn('v2', plus_one(wikiCSVDF.id)))

In [0]:
# Pandas (and Arrow) are one solution to increasing 'Normal' UDF Performance (2 of 2)
# Panda UDF to increment all values by 1
# NOTE: Pandas is especically fast on doing Calculations

from pyspark.sql.functions import pandas_udf, PandasUDFType                                                                                                                                                                                                                                                                
@pandas_udf('long', PandasUDFType.SCALAR)
def pandas_plus_one(v):
    # `v` is a pandas Series
    return v + 1  # outputs a pandas Series

# Output prior to UDF
df = spark.range(10000000000000000)
 
# Output using the UDF to increment all values by 1  
display(wikiCSVDF.withColumn('v2', pandas_plus_one(wikiCSVDF.id)))

## Comparing Higher-Order functions to Pandas UDFs to Vanilla Python UDFs

In [0]:
# Here's our 500 Line Array of data
df = (spark.createDataFrame(
         [(1, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
   (10, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe'])], 
     ["id", "words"]
))
      

df.printSchema()

In [0]:
df = df.union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df).union(df)
df.count()

In [0]:
# Using Higher-order functions 'transform' (1 of 3)
from pyspark.sql.functions import *

display(df.withColumn("tag_words", transform("words", lambda x: length(x)* 456)))


In [0]:
# Pandas UDF (2 of 3)
from pyspark.sql.functions import pandas_udf, PandasUDFType  

@pandas_udf("array<int>")
def pandasUDF(tags):
  return tags.map(lambda x: [len(words) for words in x])

display(df.withColumn("tag_words", pandasUDF("words")))

In [0]:
# Vanilla Python UDF (3 of 3)
# Too small a Dataset to have Pandas UDF out-perform the Vanilla Python UDF
@udf("array<int>")
def pythonUDF(tags):
  return [len(words) for words in tags]

display(df.withColumn("tag_words", pythonUDF("words")))

# End of Mod-03b-SparkSQL (Transformations 02)
## Ignore past here

## Compare Scala UDF to Python UDF

In [0]:
# This is newer lab than one you have
display(dbutils.fs.ls("dbfs:/databricks-datasets/asa/airlines/2007.csv"))

path,name,size
dbfs:/databricks-datasets/asa/airlines/1987.csv,1987.csv,127162942
dbfs:/databricks-datasets/asa/airlines/1988.csv,1988.csv,501039472
dbfs:/databricks-datasets/asa/airlines/1989.csv,1989.csv,486518821
dbfs:/databricks-datasets/asa/airlines/1990.csv,1990.csv,509194687
dbfs:/databricks-datasets/asa/airlines/1991.csv,1991.csv,491210093
dbfs:/databricks-datasets/asa/airlines/1992.csv,1992.csv,492313731
dbfs:/databricks-datasets/asa/airlines/1993.csv,1993.csv,490753652
dbfs:/databricks-datasets/asa/airlines/1994.csv,1994.csv,501558665
dbfs:/databricks-datasets/asa/airlines/1995.csv,1995.csv,530751568
dbfs:/databricks-datasets/asa/airlines/1996.csv,1996.csv,533922363


In [0]:
%scala
// This is newer lab then one you have

val wikiCSVDF = (spark.read
                  .option("sep", ",")
                  .csv("dbfs:/databricks-datasets/asa/airlines/1987.csv") 
            )

wikiCSVDF.foreach(_ =>())    // Use a do-nothing Custom action to establish a benchmark

wikiCSVDF: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 27 more fields]

In [0]:
# This is newer lab then one you have (Compare Clock time to Scala)

wikiCSVDF = (spark.read
                  .option("sep", ",")
                  .csv("dbfs:/databricks-datasets/asa/airlines/1987.csv") 
            )

wikiCSVDF.foreach(lambda x: None)    # Use a do-nothing Custom action to establish a baseline

In [0]:
%scala
// Here"s our 500 Line Array of data
val arraydata = Seq(
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")),
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")),
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")),
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")),
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")),
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe")), 
Row(1,List("settlementEast", "New", "Market", "now", "a", "town", "located", "always", "Dorchester", "County", "Maryland", "The", "population", "was", "400", "at", "the", "2010", "census", "The", "zip", "code", "maybe"))
)

In [0]:
%scala
import org.apache.spark.sql._
import org.apache.spark.sql.types._


val arraySchema = new StructType().add("id", IntegerType).add("words", ArrayType(StringType))

In [0]:
%scala
val df = spark.createDataFrame(
   spark.sparkContext.parallelize(arraydata),arraySchema)

In [0]:
%scala
// New one with Multiplication
def tag_lengths(tags: Seq[String]): Seq[Int] = {
  tags.map(tag => tag.length * 456)
}
val scalaUDF = udf(tag_lengths(_:Seq[String]): Seq[Int])

display(df.withColumn("tag_words", scalaUDF($"words")))

In [0]:
# Here's same data in Python
# Here's our 500 Line Array of data
df = (spark.createDataFrame(
         [(1, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (5, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (6, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (7, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (8, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (9, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
           (2, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (3, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe']), 
  (4, ['settlementEast', 'New', 'Market', 'now', 'a', 'town', 'located', 'always', 'Dorchester', 'County', 'Maryland', 'The', 'population', 'was', '400', 'at', 'the', '2010', 'census', 'The', 'zip', 'code', 'maybe'])],
     ["id", "words"]
))

df.printSchema()

In [0]:
# Vanilla Python UDF (2 of 2)
# New one with Multiplication
@udf("array<int>")
def pythonUDF(tags):
  return [len(words) * 456 for words in tags]

display(df.withColumn("tag_words", pythonUDF("words")))

In [0]:
# Vanilla Python UDF (2 of 2)
@udf("array<int>")
def pythonUDF(tags):
  return [len(words) for words in tags]

display(df.withColumn("tag_words", pythonUDF("words")))